In [ ]:
import sys, time
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d.axes3d import Axes3D
import numpy as np

from pykern import pkcli
from pykern.pkcollections import PKDict

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser
from rslaser.elements import drift, crystalSlice
from rslaser.rspulse import pulse

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

In [ ]:
"""CalcIntFromElecField(_arI, _inWfr, _inPol, _inIntType, _inDepType, _inE, _inX, _inY)
function calculates/"extracts" Intensity from pre-calculated Electric Field
:param _arI: output resulting Intensity array (should be allocated in Python script before calling this function)
:param _inWfr: input pre-calculated Wavefront structure (instance of SRWLWfr)
:param _inPol: input switch specifying polarization component to be extracted:
               =0 -Linear Horizontal; 
               =1 -Linear Vertical; 
               =2 -Linear 45 degrees; 
               =3 -Linear 135 degrees;
               =4 -Circular Right; 
               =5 -Circular Left; 
               =6 -Total
:param _inIntType: input switch specifying "type" of a characteristic to be extracted:
               =0 -"Single-Electron" Intensity; 
               =1 -"Multi-Electron" Intensity; 
               =2 -"Single-Electron" Flux; 
               =3 -"Multi-Electron" Flux; 
               =4 -"Single-Electron" Radiation Phase; NOTE: requires DOUBLE PRECISION array via srwlib.array('d', ...)
               =5 -Re(E): Real part of Single-Electron Electric Field;
               =6 -Im(E): Imaginary part of Single-Electron Electric Field;
               =7 -"Single-Electron" Intensity, integrated over Time or Photon Energy (i.e. Fluence)
               =8 -"Single-Electron" Mutual Intensity (i.e. E(r)E*(r')) 
:param _inDepType: input switch specifying type of dependence to be extracted:
               =0 -vs e (photon energy or time);
               =1 -vs x (horizontal position or angle);
               =2 -vs y (vertical position or angle);
               =3 -vs x&y (horizontal and vertical positions or angles);
               =4 -vs e&x (photon energy or time and horizontal position or angle);
               =5 -vs e&y (photon energy or time and vertical position or angle);
               =6 -vs e&x&y (photon energy or time, horizontal and vertical positions or angles);
:param _inE: input photon energy [eV] or time [s] to keep fixed (to be taken into account for dependences vs x, y, x&y)
:param _inX: input horizontal position [m] to keep fixed (to be taken into account for dependences vs e, y, e&y)
:param _inY: input vertical position [m] to keep fixed (to be taken into account for dependences vs e, x, e&x)
"""
import srwlib
from srwlib import srwl

In [ ]:
# define a simple lattice

dr1 = drift.drift('dr1', 0.2)                      # 20 cm drift
crystal = crystalSlice.crystalSlice('cryst_slice1', 0.1)  # 10 cm single-slice crystal 
dr2 = drift.drift('dr2', 0.02)                     #  2 cm drift
lattice = [(dr1,'default'), (crystal,'abcd'), (dr2,'default')]
  
current_position = 0.0 
print('Initial z position of the pulse:', current_position, 'm')

In [ ]:
# Specify all top-level input parameters
a0 = 0.01            # 0.85e-9 * lambda [microns] * Sqrt(Intensity [W/cm^2])
lambda0 = 8.e-7      # wavelength [m]
tau_fwhm = 0.3e-13   # FWHM pulse length [s]
w0 = 400.e-6         # waist size [m]
z_waist = 0.0        # longitudinal waist position [m]
z_center = 0.0       # longitudinal position of pulse center [m]

d_lambda = 0.0     # full wavelength chirp across laser pulse [m]
dw0x = 0.0           # deviation of horizontal waist from w0
dw0y = 0.0           # deviation of vertical waist from w0
dzwx = 0.0           # deviation of horizontal waist position from z_waist
dzwy = 0.0           # deviation of vertical waist position from z_waist
x_shift = 0.         # bulk horizontal shift of pulse position
y_shift = 0.         # bulk vertical shift of pulse position

#  Pulse parameters: 
params = PKDict(
    propLen = 15.0  # [m] 
)
params.sigrW = 0.000186  # 0.000436984 #  radial rms size at the waist [m] 
params.d_to_w = 0.01  #  [m] distance from the initial pulse location to the loc-n of the beam waist, > 0 if converging 
params.pulseE = 0.001  # pulse energy [J] 
params.poltype = 1  #  0 = linear horizontal, 1 = linear vertical, ... 
params.phE = 1.55  # Wavefront energy [eV]. 1.55 eV is 800 nm wavelength (seed laser); 532 nm for the pump laser 
params.energyChirp = 0.0
params.nslice = 5  #  the number of slices the pulse is divided into 

params.a0 = 1.
params.lambda0 = lambda0
params.tau_fwhm = tau_fwhm
params.w0 = w0
params.z_waist = z_waist
params.z_center = z_center

params.d_lambda = d_lambda
params.dw0x = dw0x
params.dw0y = dw0y
params.dzwx = dzwx
params.dzwy = dzwy
params.x_shift = x_shift
params.y_shift = y_shift

# Instantiate the pulse
thisPulse = pulse.LaserPulse(params)

In [ ]:
# initial pulse - intensity

# choose one of the laser pulse slices, and grab its SRW wavefront object
slice_array=thisPulse.slice
#slice_number = int(int(params.nslice-1)/2)
slice_number = 1
# print('slice_number = ', slice_number)
wfr0=slice_array[slice_number].wfr

arIinit = srwlib.array('f', [0]*wfr0.mesh.nx*wfr0.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIinit, wfr0, 0, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
arIinit2D = np.array(arIinit).reshape((wfr0.mesh.nx, wfr0.mesh.ny), order='C')
wfrsizei=np.size(arIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvalsf02=np.linspace(wfr0.mesh.xStart,wfr0.mesh.xFin,wfr0.mesh.nx)
yvalsf02=np.linspace(wfr0.mesh.yStart,wfr0.mesh.yFin,wfr0.mesh.ny)


In [ ]:
# plot the computed intensity

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(12,7))
    ax = fig.gca()

    # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
    plt.pcolormesh(np.multiply(xvalsf02,1e6), np.multiply(yvalsf02,1e6), arIinit2D, cmap=plt.cm.viridis,shading='auto')
    plt.colorbar()
    #['Horizontal Position [mm]', , 'Intensity']
    ax.set_ylabel(r'Vertical Position [$\mu m$]')
    ax.set_xlabel(r'Horizontal Position [$\mu m$]')
    ax.set_title('Intensity before the crystal')

In [ ]:
# initial pulse - phase
# wfr1=slice_array[slice_number].wfr
wfr1=slice_array[1].wfr

brIinit = srwlib.array('d', [0]*wfr1.mesh.nx*wfr1.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(brIinit, wfr1, 0, 4, 3, wfr1.mesh.eStart, 0, 0) #extracts the phase; must use double precision

##Reshaping electric field data from flat to 2D array
brIinit2D = np.array(brIinit).reshape((wfr1.mesh.nx, wfr1.mesh.ny), order='C')
wfrsize=np.size(brIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(brIinit2D))
xvalsf02=np.linspace(wfr1.mesh.xStart,wfr1.mesh.xFin,wfr1.mesh.nx)
yvalsf02=np.linspace(wfr1.mesh.yStart,wfr1.mesh.yFin,wfr1.mesh.ny)

In [ ]:
# plot the phase

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(12,7))
    ax = fig.gca()

    # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
    plt.pcolormesh(np.multiply(xvalsf02,1e6), np.multiply(yvalsf02,1e6), brIinit2D, cmap=plt.cm.viridis,shading='auto')
    plt.colorbar()
    #['Horizontal Position [mm]', , 'Phase']
    ax.set_ylabel(r'Vertical Position [$\mu m$]')
    ax.set_xlabel(r'Horizontal Position [$\mu m$]')
    ax.set_title('Phase before the crystal')

In [ ]:
# plot the phase in 3D
fig = plt.figure(figsize=(28,12))
# here, `ax` is a 3D-aware axis instance because of the projection='3d' keyword
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Phase before the crystal')
p = ax.plot_surface(2.*np.multiply(xvalsf02,1e4), 2.*np.multiply(yvalsf02,1e4), brIinit2D, 
                    rstride=40, cstride=40, cmap=plt.cm.viridis, linewidth=0, antialiased=False)
cb = fig.colorbar(p, shrink=0.5)

In [ ]:
#  Propagate the pulse through the optical beamline: 
for i in lattice:
    current_elem, prop_type = i 
    # print (current_elem, prop_type) 
    thisPulse = current_elem.propagate(thisPulse, prop_type)
    current_position += current_elem.length
    print('Current position in the beamline:', current_position, ' m')

In [ ]:
# transmitted pulse - intensity
wfr2=slice_array[slice_number].wfr

arIinit = srwlib.array('f', [0]*wfr2.mesh.nx*wfr2.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(arIinit, wfr2, 0, 0, 3, wfr0.mesh.eStart, 0, 0) #extracts intensity

##Reshaping electric field data from flat to 2D array
arIinit2D = np.array(arIinit).reshape((wfr2.mesh.nx, wfr2.mesh.ny), order='C')
wfrsizei=np.size(arIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(arIinit2D))
xvalsf02=np.linspace(wfr2.mesh.xStart,wfr2.mesh.xFin,wfr2.mesh.nx)
yvalsf02=np.linspace(wfr2.mesh.yStart,wfr2.mesh.yFin,wfr2.mesh.ny)

In [ ]:
# plot the computed intensity

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(12,7))
    ax = fig.gca()

    # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
    plt.pcolormesh(np.multiply(xvalsf02,1e6), np.multiply(yvalsf02,1e6), arIinit2D, cmap=plt.cm.viridis,shading='auto')
    plt.colorbar()
    #['Horizontal Position [mm]', , 'Intensity']
    ax.set_ylabel(r'Vertical Position [$\mu m$]')
    ax.set_xlabel(r'Horizontal Position [$\mu m$]')
    ax.set_title('Intensity before the crystal')

In [ ]:
# transmitted pulse - phase
#wfr3=slice_array[slice_number].wfr
wfr3=slice_array[2].wfr

brIinit = srwlib.array('d', [0]*wfr3.mesh.nx*wfr3.mesh.ny) # "flat" array to take 2D intensity data
srwl.CalcIntFromElecField(brIinit, wfr3, 0, 4, 3, wfr3.mesh.eStart, 0, 0) #extracts the phase; must use double precision

##Reshaping electric field data from flat to 2D array
brIinit2D = np.array(brIinit).reshape((wfr3.mesh.nx, wfr3.mesh.ny), order='C')
wfrsize=np.size(brIinit)

print('Size of initial wavefront data array (coordinate):',np.shape(brIinit2D))
xvalsf02=np.linspace(wfr3.mesh.xStart,wfr3.mesh.xFin,wfr3.mesh.nx)
yvalsf02=np.linspace(wfr3.mesh.yStart,wfr3.mesh.yFin,wfr3.mesh.ny)

In [ ]:
# plot the phase

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(12,7))
    ax = fig.gca()

    # By setting wfr0a.unitElFldAng=1, default units should  now be in mrad(?)
    plt.pcolormesh(np.multiply(xvalsf02,1e6), np.multiply(yvalsf02,1e6), brIinit2D, cmap=plt.cm.viridis,shading='auto')
    plt.colorbar()
    #['Horizontal Position [mm]', , 'Phase']
    ax.set_ylabel(r'Vertical Position [$\mu m$]')
    ax.set_xlabel(r'Horizontal Position [$\mu m$]')
    ax.set_title('Phase after the crystal')

In [ ]:
# plot the phase in 3D
fig = plt.figure(figsize=(28,12))
# here, `ax` is a 3D-aware axis instance because of the projection='3d' keyword
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Phase before the crystal')
p = ax.plot_surface(np.multiply(xvalsf02,1e4), np.multiply(yvalsf02,1e4), brIinit2D, 
                    rstride=40, cstride=40, cmap=plt.cm.viridis, linewidth=0, antialiased=False)
cb = fig.colorbar(p, shrink=0.5)